In [4]:
import pandas as pd
import numpy as np

# import the required classes and definitions from Python-MIP
from mip import Model, xsum, maximize, BINARY, INTEGER

Using Python-MIP package version 1.6.6


In [5]:
#sort the top500 by element_type so model can fill correct number of each type

top500 = pd.read_csv('top500.csv')
top500sorted = top500.sort_values('element_type')
fitted = pd.read_csv('fitted')
top500sorted = top500sorted.merge(fitted, how='left', on='id_')
top500sorted.fillna(0, inplace=True)

#lists for the dream team model, sorted by position
eltype = list(top500sorted['element_type'])
form = list(top500sorted['form'])
score = list(top500sorted['total_points'])
price = list(top500sorted['now_cost'])
team = list(top500sorted['team'])
fitted = list(top500sorted['fitted'])
form_score = list(pd.Series(form) * pd.Series(score))


In [6]:
#make a list of our main columns of interest
simple_columns = ['web_name', 'element_type', 'total_points', 'fitted', 'now_cost', 'form',
                 'goals_scored', 'assists', 'clean_sheets', 'team', 'selected_by_percent']

#make a smaller df for easy viewing of dream_team
top500simple = top500sorted[simple_columns]
top500simple.shape

(500, 11)

In [7]:
#find cutoffs for the element_types

eltype_counts = top500sorted['element_type'].value_counts().sort_index()

cutoff1 = eltype_counts[1]
cutoff2 = cutoff1 + eltype_counts[2]
cutoff3 = cutoff2 + eltype_counts[3]
cutoff4 = cutoff3 + eltype_counts[4]

print(cutoff1, cutoff2, cutoff3, cutoff4)
eltype_counts

44 219 431 500


1     44
2    175
3    212
4     69
Name: element_type, dtype: int64

In [8]:
df = top500sorted
type(df)

pandas.core.frame.DataFrame

In [9]:
df = df.reset_index()
df.head()

,index,id_,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,...,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards,fitted
0,0,450,0,1,120,0.0,0.0,0,40694,0,...,17,63791,133,53310,1191,0.1,3.9,Roberto,0,0.819393
1,1,281,0,2,46,0.0,0.0,0,18499,0,...,11,296051,3953,166660,3848,0.0,2.8,McGovern,0,0.667807
2,2,494,1,10,337,0.0,0.0,4,225321,0,...,70,425639,46476,135350,1457,0.8,15.2,Ramsdale,0,3.990377
3,3,436,0,0,9,0.0,0.0,0,74854,0,...,1,2839,88,4831,119,0.0,0.2,Moore,0,0.415165
4,4,131,0,10,304,100.0,100.0,6,40836,0,...,69,200253,18104,59231,3110,1.4,13.5,Guaita,1,3.730542


In [10]:
ars = list(df[df.team == 1].index)
avl = list(df[df.team == 2].index)
bou = list(df[df.team == 3].index)
bha = list(df[df.team == 4].index)
bur = list(df[df.team == 5].index)
che = list(df[df.team == 6].index)
cry = list(df[df.team == 7].index)
eve = list(df[df.team == 8].index)
lei = list(df[df.team == 9].index)
liv = list(df[df.team == 10].index)
mci = list(df[df.team == 11].index)
mun = list(df[df.team == 12].index)
new = list(df[df.team == 13].index)
nor = list(df[df.team == 14].index)
shu = list(df[df.team == 15].index)
sou = list(df[df.team == 16].index)
tot = list(df[df.team == 17].index)
wat = list(df[df.team == 18].index)
whu = list(df[df.team == 19].index)
wol = list(df[df.team == 20].index)

In [11]:
# The model

# define the problem data, change the profit list here
profit = score #run with current_week_points and compare
weight = price
c = 830
n = len(weight)

#create an empty maximization
m = Model('team_select')

#add the binary decision variables to model m and store their references in a list x
x = [m.add_var(var_type=BINARY) for i in range(n)]

#define the objective function of this model 
m.objective = maximize(xsum(profit[i] * x[i] for i in range(n)))

#add the capacity constraints
m += xsum(weight[i] * x[i] for i in range(n)) <= c

m += xsum(x) <= 11

m += xsum(eltype[i] * x[i] for i in range(cutoff1)) >= 1
m += xsum(eltype[i] * x[i] for i in range(cutoff1)) <= 1

m += xsum(eltype[i] * x[i] for i in range(cutoff1,cutoff2)) >= 6
m += xsum(eltype[i] * x[i] for i in range(cutoff1,cutoff2)) <= 10

m += xsum(eltype[i] * x[i] for i in range(cutoff2,cutoff3)) >= 6
m += xsum(eltype[i] * x[i] for i in range(cutoff2,cutoff3)) <= 15

m += xsum(eltype[i] * x[i] for i in range(cutoff3,cutoff4)) >= 4
m += xsum(eltype[i] * x[i] for i in range(cutoff3,cutoff4)) <= 12


# max 3 per team

m += xsum(df.team[i] * x[i] for i in ars) <= 3
m += xsum(df.team[i] * x[i] for i in avl) <= 6
m += xsum(df.team[i] * x[i] for i in bou) <= 9
m += xsum(df.team[i] * x[i] for i in bha) <= 12
m += xsum(df.team[i] * x[i] for i in bur) <= 15
m += xsum(df.team[i] * x[i] for i in che) <= 18
m += xsum(df.team[i] * x[i] for i in cry) <= 21
m += xsum(df.team[i] * x[i] for i in eve) <= 24
m += xsum(df.team[i] * x[i] for i in lei) <= 27
m += xsum(df.team[i] * x[i] for i in liv) <= 30
m += xsum(df.team[i] * x[i] for i in mci) <= 33
m += xsum(df.team[i] * x[i] for i in mun) <= 36
m += xsum(df.team[i] * x[i] for i in new) <= 39
m += xsum(df.team[i] * x[i] for i in nor) <= 42
m += xsum(df.team[i] * x[i] for i in shu) <= 45
m += xsum(df.team[i] * x[i] for i in sou) <= 48
m += xsum(df.team[i] * x[i] for i in tot) <= 51
m += xsum(df.team[i] * x[i] for i in wat) <= 54
m += xsum(df.team[i] * x[i] for i in whu) <= 57
m += xsum(df.team[i] * x[i] for i in wol) <= 60



#Optimize the model
m.optimize()

# Compute the solution, a list of the selected items
selected = [i for i in range(n) if x[i].x >= 0.99]
print('selected items: {}'.format(selected))
dream_team = top500simple.iloc[selected,:]
dream_team

selected items: [2, 158, 161, 164, 169, 378, 396, 397, 441, 442, 499]


,web_name,element_type,total_points,fitted,now_cost,form,goals_scored,assists,clean_sheets,team,selected_by_percent
2,Ramsdale,1,70,3.990377,46,3.6,0,1,4,3,4.7
158,Lundstram,2,89,4.233223,52,3.8,3,3,6,15,48.5
161,Pereira,2,85,4.453372,65,4.8,2,1,7,9,17.3
164,Baldock,2,77,5.077554,49,5.2,2,3,6,15,7.6
169,van Dijk,2,73,3.750909,64,6.6,3,0,4,10,39.9
378,Maddison,3,88,4.888151,78,5.2,5,5,6,9,27.8
396,Mané,3,115,3.143700,123,6.4,9,7,4,10,39.9
397,De Bruyne,3,116,4.646223,103,8.0,6,11,7,11,40.6
441,Abraham,4,99,4.941112,80,3.6,11,3,4,6,36.8
442,Pukki,4,93,4.505445,66,6.4,9,3,2,14,17.0


In [12]:
sum(dream_team['now_cost'])

827

In [13]:
sum(dream_team['total_points'])

1038